<a href="https://colab.research.google.com/github/Damilola-ops/Playlist_Generator/blob/main/Playlist_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentence_transformers
!pip install datasets 
!pip install transformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 30.9 MB/s 
     |████████████████████████████████| 57 kB 7.4 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 270 kB 75.5 MB/s 
     |████████████████████████████████| 212 kB 80.0 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 2.3 MB 58.1 MB/s 
     |████████████████████████████████| 112 kB 74.2 MB/s 
     |████████████████████████████

In [ ]:
config = {
    'Name':'Semantic Search with Sentence embeddings',
    'dataset':'100 samples with songs and verses no lyrics',
    'Frontend':'Gradio'}

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

In [4]:
import transformers
import gradio as gr
from huggingface_hub import hf_hub_download
from sentence_transformers import SentenceTransformer,util
from datasets import load_dataset
import pickle
import pandas as pd

In [ ]:
wandb.init(project="Playlist Generator", entity="damilojohn",config = config)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damilojohn. Use `wandb login --relogin` to force relogin


In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/NimaBoscarino/playlist-generator

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'playlist-generator'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [5]:
songs_df = pd.read_csv(hf_hub_download('NimaBoscarino/playlist-generator',repo_type='dataset',filename='songs_new.csv'))
verses_df = pd.read_csv(hf_hub_download('NimaBoscarino/playlist-generator',repo_type='dataset',filename='verses.csv'))

Downloading:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

In [6]:
song_embeddings = pickle.load(open(hf_hub_download("NimaBoscarino/playlist-generator", repo_type="dataset", filename="verse-embeddings.pkl"), "rb"))

Downloading:   0%|          | 0.00/252M [00:00<?, ?B/s]

In [7]:
len(song_embeddings)

164057

In [8]:
embedder = SentenceTransformer('msmarco-MiniLM-L-6-v3')

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [9]:
def generate_playlist(prompt):
  prompt_embedding = embedder.encode(prompt)
  hits = util.semantic_search(prompt_embedding,song_embeddings,top_k=20)
  hits = pd.DataFrame.from_dict(hits[0])
  verses_match = verses_df.iloc[hits['corpus_id']]
  verses_match = verses_df.drop_duplicates(subset=['song_id'])
  songs_match = songs_df[songs_df['song_id'].isin(verses_match['song_id'].values)]
  songs_match.song_id = pd.Categorical(songs_match.song_id,categories=verses_match['song_id'].values)
  songs_match = songs_match.sort_values('song_id')
  songs_match = songs_match.loc[0:9]
  songs_name = list(songs_match['full_title'])
  songs_art = list(songs_match['art'].fillna("https://i.imgur.com/bgCDfT1.jpg"))
  images = [gr.Image.update(value=art,visible=True) for art in songs_art]
  return (
      gr.Radio.update(label='songs',interactive=True,choices=songs_name),*images

  )
      
  



In [10]:
def set_lyrics(full_title):
  lyrics_text = lyrics[lyrics['song_id'].isin(songs_df[songs_df['full_title'] == full_title]['song_id'])]['text']
  return lyrics_text

In [11]:
prompt = 'I think I want to die ' 
hits = generate_playlist(prompt)
hits

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


({'choices': ['Remember That Night? by\xa0Sara\xa0Kays',
   'Chosen Last by\xa0Sara\xa0Kays',
   'Everything Will Be Alright by\xa0The\xa0Killers',
   'Smaller Than This by\xa0Sara\xa0Kays',
   'Francine by\xa0ZZ\xa0Top',
   'Hallelujah by\xa0HAIM',
   'Work from Home by\xa0Fifth\xa0Harmony (Ft.\xa0Ty\xa0Dolla $ign)',
   'Body Bag by\xa0Charlotte\xa0Lawrence',
   'This River is Wild by\xa0The\xa0Killers',
   'Seasons by\xa06LACK (Ft.\xa0Khalid)',
   'Home for the Summer by\xa0Sara\xa0Kays',
   'Precious and Grace by\xa0ZZ\xa0Top',
   'BBC by\xa0JAY-Z (Ft.\xa0Beyoncé, Justin\xa0Timberlake, Nas & Swizz\xa0Beatz)',
   'Quiet Town by\xa0The\xa0Killers',
   'Mary Had a Baby by\xa0Hall\xa0& Oates',
   'Worst Case Scenario by\xa0Kid\xa0Quill',
   'Manic Mechanic by\xa0ZZ\xa0Top',
   'Use This Gospel by\xa0Kanye\xa0West (Ft.\xa0Clipse & Kenny\xa0G)',
   'Closure by\xa0Hayd',
   'My Song 5 by\xa0HAIM',
   'Doubleback by\xa0ZZ\xa0Top',
   'Interlude by\xa0Madison\xa0Beer',
   'Worth It by\xa0Fif

In [12]:
def set_example_prompt(examples):
  return gr.TextArea.update(value=examples[0])


In [13]:
demo = gr.Blocks()
with demo:
    gr.Markdown(
        """ 
        # Playlist Generator 🎶🎧🎵
        """)

    with gr.Row():
        with gr.Column():
            gr.Markdown(
                ''' 
                Enter a prompt and generate a playlist based on the prompt 
                This was built using sentence transformers and Gradio
                ''')
            song_prompt = gr.TextArea(
                value = 'Running wild and free',
                placeholder = 'Enter a song prompt, or choose an example'
            )
            example_prompts = gr.Dataset(
                components=[song_prompt],
                samples=[
                    ["Running wild and free"],
                    ["I'm deeply in love with someone I just met!"],
                    ["My friends mean the world to me"],
                    ["Sometimes I feel like no one understands"],
                ]
            )
        with gr.Column():
            fetch_songs = gr.Button(value="Generate Your Playlist 🧑🏽‍🎤").style(full_width=True)
            with gr.Row():
                tile1 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile2 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile3 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
            with gr.Row():
                tile4 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile5 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile6 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
            with gr.Row():
                tile7 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile8 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile9 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)

            # Workaround because of the Gallery issues
            tiles = [tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

            song_option = gr.Radio(label="Songs", interactive=True, choices=None, type="value")

        with gr.Column():
            verse = gr.Textbox(label="Verse", placeholder="Select a song to see its lyrics")
    fetch_songs.click(
        fn=generate_playlist,
        inputs=[song_prompt],
        outputs = [song_option,*tiles]
    )
    example_prompts.click(
        fn=set_example_prompt,
        inputs=[example_prompts],
        outputs= example_prompts.components
    )
  
          
    
                
demo.launch(debug=True) 
                    
                                         
            

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://18089.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 276, in run_predict
    fn_index, raw_input, username, session_state, iterators
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 789, in process_api
    predictions = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 754, in postprocess_data
    postprocess=block_fn.postprocess,
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 341, in postprocess_update_dict
    prediction_value = block.g

Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f3d989055d0>,
 'http://127.0.0.1:7860/',
 'https://18089.gradio.app')